In [72]:
import pandas as pd 
import numpy as np
import yfinance
from IPython.display import HTML
import random
#from ipynb.fs.full.Functions1 import hide_toggle
#from operator import itemgetter 
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
%matplotlib inline
#from mpl_finance import candlestick_ohlc
from mplfinance.original_flavor import candlestick_ohlc
import pandas_datareader as pdr
from operator import itemgetter
import datetime as dt
import talib as ta
from itertools import compress
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.signal
import math
from math import sqrt
# import warnings
# warnings.simplefilter("ignore")
import math
from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

In [2]:
symbol ="GC=F"
num_name= symbol.split(' ') 
df = []
for i in num_name:
        ticker = yfinance.Ticker(i)
        data=ticker.history(interval='60m',period="3mo")
        #data['Date'] = pd.to_datetime(data.index) # just to crate another column for date!!!
        #data['Date'] = data['Date'].apply(mpl_dates.date2num)
        data = data.loc[:,['Open', 'High', 'Low', 'Close']]
#         ohlc_data= data.loc[:,['Date', 'Open', 'High', 'Low', 'Close']].values

        df.append(data)

df[0]

,Open,High,Low,Close
Datetime,,,,
2020-11-19 16:00:00-05:00,1865.400024,1866.699951,1864.400024,1864.500000
2020-11-19 18:00:00-05:00,1864.800049,1864.900024,1859.099976,1861.500000
2020-11-19 19:00:00-05:00,1861.699951,1863.400024,1859.599976,1861.699951
2020-11-19 20:00:00-05:00,1861.699951,1866.500000,1860.900024,1863.900024
2020-11-19 21:00:00-05:00,1863.800049,1864.400024,1861.000000,1863.000000
...,...,...,...,...
2021-02-19 13:00:00-05:00,1782.000000,1782.099976,1775.900024,1779.000000
2021-02-19 14:00:00-05:00,1779.199951,1780.000000,1777.699951,1779.099976
2021-02-19 15:00:00-05:00,1779.000000,1781.699951,1778.400024,1781.099976


In [3]:
Data=[]
for i in range(len(num_name)):
    Data.append(df[i].iloc[:].values)
len(Data)

1

In [5]:
pts=[]
for i in range(len(num_name)):
    month_diff = df[i]["Close"].shape[0] // (30*7) # Integer divide the number of prices we have by 30 # for HOURLY
    if month_diff == 0: # We want a value greater than 0
        month_diff = 1
    smooth = int(2 * month_diff + 3) # Simple algo to determine smoothness
    pts.append(savgol_filter(df[i]["Close"], smooth, 3)) # Get the smoothened price data
len(pts)

1

In [6]:
for i in range(len(num_name)):
    df[i]["PTS"]= pts[i]


In [7]:
def pythag(pt1, pt2):
    a_sq = (pt2[0] - pt1[0]) ** 2
    b_sq = (pt2[1] - pt1[1]) ** 2
    return sqrt(a_sq + b_sq)

def local_min_max(pts):
    local_min = []
    local_max = []
    prev_pts = [(0, pts[0]), (1, pts[1])]
    for i in range(1, len(pts) - 1):
        append_to = ''
        if pts[i-1] > pts[i] < pts[i+1]:
            append_to = 'min'
        elif pts[i-1] < pts[i] > pts[i+1]:
            append_to = 'max'
        if append_to:
            if local_min or local_max:
                prev_distance = pythag(prev_pts[0], prev_pts[1]) * 0.5
                curr_distance = pythag(prev_pts[1], (i, pts[i]))
                if curr_distance >= prev_distance:
                    prev_pts[0] = prev_pts[1]
                    prev_pts[1] = (i, pts[i])
                    if append_to == 'min':
                        local_min.append((i, pts[i]))
                    else:
                        local_max.append((i, pts[i]))
            else:
                prev_pts[0] = prev_pts[1]
                prev_pts[1] = (i, pts[i])
                if append_to == 'min':
                    local_min.append((i, pts[i]))
                else:
                    local_max.append((i, pts[i]))
    return local_min, local_max

In [8]:
local_min_max1=[]
for i in range(len(num_name)):
    local_min_max1.append(local_min_max(pts[i]))
len(local_min_max1)

1

In [10]:
df[0]

,Open,High,Low,Close,PTS
Datetime,,,,,
2020-11-19 16:00:00-05:00,1865.400024,1866.699951,1864.400024,1864.500000,1862.604624
2020-11-19 18:00:00-05:00,1864.800049,1864.900024,1859.099976,1861.500000,1862.876621
2020-11-19 19:00:00-05:00,1861.699951,1863.400024,1859.599976,1861.699951,1863.227683
2020-11-19 20:00:00-05:00,1861.699951,1866.500000,1860.900024,1863.900024,1863.631215
2020-11-19 21:00:00-05:00,1863.800049,1864.400024,1861.000000,1863.000000,1864.060620
...,...,...,...,...,...
2021-02-19 13:00:00-05:00,1782.000000,1782.099976,1775.900024,1779.000000,1782.823898
2021-02-19 14:00:00-05:00,1779.199951,1780.000000,1777.699951,1779.099976,1782.637850
2021-02-19 15:00:00-05:00,1779.000000,1781.699951,1778.400024,1781.099976,1781.920617


In [16]:
df[0]['Close']

Datetime
2020-11-19 16:00:00-05:00    1864.500000
2020-11-19 18:00:00-05:00    1861.500000
2020-11-19 19:00:00-05:00    1861.699951
2020-11-19 20:00:00-05:00    1863.900024
2020-11-19 21:00:00-05:00    1863.000000
                                ...     
2021-02-19 13:00:00-05:00    1779.000000
2021-02-19 14:00:00-05:00    1779.099976
2021-02-19 15:00:00-05:00    1781.099976
2021-02-19 16:00:00-05:00    1780.699951
2021-02-19 16:45:31-05:00    1781.099976
Name: Close, Length: 1381, dtype: float64

In [40]:
# create 20 days simple moving average column
df[0]['20_SMA'] = c.rolling(window = 10, min_periods = 1).mean()
# create 50 days simple moving average column
df[0]['50_SMA'] = b.rolling(window = 15, min_periods = 1).mean()
# display first few rows
list(df[0]['20_SMA'])

AttributeError: 'list' object has no attribute 'rolling'

In [26]:
a=pd.DataFrame(local_min_max1[0][0])

b=pd.DataFrame(local_min_max1[0][1])

In [70]:
c=[a[1] for a in local_min_max1[0][0]]
#local_min_max1[0][1]

In [100]:
numbers = c
window_size = 5

numbers_series = pd.Series(numbers)
windows = numbers_series.rolling(window_size)
moving_averages = windows.mean()

moving_averages_list = moving_averages.tolist()
without_nans = moving_averages_list[window_size - 1:]

without_nans

[1818.447983906614,
 1819.0753697902041,
 1831.5401718926855,
 1843.1988168348876,
 1842.7363144929052,
 1841.1164920858773,
 1845.811241736789,
 1850.3583349609478,
 1859.2220989686546,
 1867.4423152264317,
 1874.6527992876945,
 1878.5587171017855,
 1877.7942821602905,
 1890.9356253977057,
 1898.4040263097534,
 1907.9081316724514,
 1915.042104337539,
 1905.6162926859554,
 1883.9675970155295,
 1865.3867943120863,
 1855.4227396997062,
 1842.4470114536302,
 1845.6815624116339,
 1848.4619595986035,
 1854.461294586054,
 1850.9116108641579,
 1849.2426130117117,
 1845.7543247136698,
 1843.4520295266648,
 1843.2440010119237,
 1841.0350060538137,
 1840.0906083189088,
 1837.3470947928552,
 1827.0016704079571,
 1824.078370709322,
 1825.5062470172968,
 1826.711228679132,
 1830.3247912542524,
 1836.9497801408827,
 1833.1951727322644,
 1822.7949190248619,
 1809.0069868407202,
 1794.7795810723733]

In [101]:
indexes= [a[0] for a in local_min_max1[0][0]]
indexes

[45,
 64,
 105,
 188,
 215,
 228,
 272,
 286,
 320,
 348,
 396,
 445,
 465,
 480,
 493,
 551,
 585,
 671,
 685,
 704,
 714,
 740,
 813,
 851,
 913,
 934,
 949,
 963,
 976,
 993,
 1004,
 1017,
 1035,
 1056,
 1099,
 1121,
 1134,
 1146,
 1217,
 1234,
 1240,
 1246,
 1264,
 1275,
 1311,
 1330,
 1361]

In [ ]:
without_nans

In [102]:
#thus, you will do rolling window -1 as the index of inxies!
a=list(zip(indexes[4:],without_nans))
a

[(215, 1818.447983906614),
 (228, 1819.0753697902041),
 (272, 1831.5401718926855),
 (286, 1843.1988168348876),
 (320, 1842.7363144929052),
 (348, 1841.1164920858773),
 (396, 1845.811241736789),
 (445, 1850.3583349609478),
 (465, 1859.2220989686546),
 (480, 1867.4423152264317),
 (493, 1874.6527992876945),
 (551, 1878.5587171017855),
 (585, 1877.7942821602905),
 (671, 1890.9356253977057),
 (685, 1898.4040263097534),
 (704, 1907.9081316724514),
 (714, 1915.042104337539),
 (740, 1905.6162926859554),
 (813, 1883.9675970155295),
 (851, 1865.3867943120863),
 (913, 1855.4227396997062),
 (934, 1842.4470114536302),
 (949, 1845.6815624116339),
 (963, 1848.4619595986035),
 (976, 1854.461294586054),
 (993, 1850.9116108641579),
 (1004, 1849.2426130117117),
 (1017, 1845.7543247136698),
 (1035, 1843.4520295266648),
 (1056, 1843.2440010119237),
 (1099, 1841.0350060538137),
 (1121, 1840.0906083189088),
 (1134, 1837.3470947928552),
 (1146, 1827.0016704079571),
 (1217, 1824.078370709322),
 (1234, 1825.506

In [103]:
#indexes[19:]

In [104]:
# L = [2,3,1,4,5]

# indices, L_sorted = zip(*sorted(enumerate(L), key=itemgetter(1)))
# list(L_sorted)
# >>> [1, 2, 3, 4, 5]
# list(indices)
# >>> [2, 0, 1, 3, 4]
# for i in range(0, len(df[0]['Close'])-1):
#     final_dataframe.loc[i, 'Line'] = 
# final_dataframe

In [105]:
def MacdPlotly(num):
    fig = go.Figure()

    fig = make_subplots(rows=2, cols=1,row_heights=[0.65, 0.35],shared_xaxes=True, vertical_spacing=0.015)
    #Candlestick
    stock_name= num_name[num]
    fig.add_trace(go.Candlestick(x=df[num].index,
                    open=Data[num][:,0],
                    high=Data[num][:,1],
                    low=Data[num][:,2],
                    close=Data[num][:,3], name = f'{stock_name} '),
                 row=1, col=1)
    fig.add_trace(go.Scatter(
            x=df[num]["Close"].index[indexes[19:]],
            y=without_nans,
            name="Not sure min",
            #fill='tonextx', fillcolor ='violet',
            #marker=dict(color="lightblue"),
            line = dict(color = 'Blue', width=2)
        ),row=1, col=1)
        
        
        
       
 # Add titles
    fig.update_layout(
        title=f'{stock_name} live share price evolution',
        yaxis_title='Stock Price (USD per Shares)',
        #autosize=True,
        #dragmode='pan'
        height=600, width=1000,
        hovermode='x unified'

        )

    # X-Axes
    fig.update_xaxes(
        rangeslider_visible=False,
        rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                #dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
            ],
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15d", step="day", stepmode="backward"),
                dict(count=1, label="1mo", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="todate"),
                dict(count=3, label="3mo", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    #fig.update_traces(xaxis="x4")
    #fig.update_traces(xaxis='x2') 

    #Show
    fig.show()

In [106]:
MacdPlotly(0)